# NLP model

## Importing main libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/usuario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data prepocessing

In [2]:
df_feelings = pd.read_excel('reviews_claro.xlsx')

In [3]:
df = df_feelings.copy()
df = df[['at', 'content','score']]
df = df.dropna(axis=0).reset_index(drop=True)
df = df.rename(columns={'at': 'date', 'content': 'review'})
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.strftime('%Y-%m-%d')

### Data tokenization

In [4]:
def tokenization(df):
    corpus = []
    for i in range(0, len(df)):
        review = re.sub('[^a-zA-ZÀ-ÿ]', ' ', df['review'][i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        all_stopwords = stopwords.words('spanish')
        all_stopwords.remove('sí')
        all_stopwords.remove('no')
        all_stopwords.remove('ni')
        review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

## Model training

In [13]:
dataset = tokenization(df)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=4000)
X = cv.fit_transform(dataset).toarray()
y = df['score'].values

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [72]:
# We use random forest classifier for easiness in application.
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators = 10, random_state=0)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)

In [73]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred) # 0.751

[[ 5411   100   131    45   370]
 [  832    53    79    23   147]
 [  557    54   116    54   427]
 [  247    30    91   123  1533]
 [  338    25    73   196 10449]]


0.7511160714285714

In [158]:
import pickle
pickle.dump(cv, open("count_vectorizer.pickle", "wb"))
pickle.dump(RF, open("RF_nlp.pickle", "wb"))

In [87]:
from pysentimiento import create_analyzer

In [89]:
analyzer = create_analyzer(task="sentiment", lang="es")

loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/tokenizer.json from cache at /Users/usuario/.cache/huggingface/transformers/47dd2d3180a6186d30715516321375322e3a84d5e4656762e083091bbb5d5dc4.0843b07596b388e054bae078721182b4846b9e28a7bbf04d7079b274f8613ae3
loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/special_tokens_map.json from cache at /Users/usuario/.cache/huggingface/transformers/25e0e805456d2786a12b70b86278c6e839d19958cb4f541ee1f78621140098f7.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/tokenizer_config.json from cache at /Users/usuario/.cache/huggingface/transformers/4fdf9a5a8e0a6023e1a9cdef62921158c3db9545e73f8dc0f46340a21bbb64d5.50a2bcf7668df2ff5a82b7

In [100]:
analyzer.predict('Nutritivo y gordo'), analyzer.predict('excelente'), analyzer.predict('NO me gustó')

(AnalyzerOutput(output=NEU, probas={NEU: 0.977, POS: 0.020, NEG: 0.004}),
 AnalyzerOutput(output=POS, probas={POS: 0.976, NEU: 0.019, NEG: 0.004}),
 AnalyzerOutput(output=NEG, probas={NEG: 0.898, NEU: 0.074, POS: 0.028}))

In [ ]:
pred_sentimientos = analyzer.predict(corpus)
output_sent = [pred.output for pred in pred_sentimientos]
vec_sent = []
for elem in output_sent:
    if elem == 'POS':
        vec_sent.append('positivo')
    elif elem == 'NEG':
        vec_sent.append('negativo')
    else:
        vec_sent.append('neutro')

In [188]:
df['feeling'] = vec_sent

In [205]:
df.to_csv('db.csv', index=False)

In [ ]:
import sys
!{sys.executable} -m pip install -e git+https://github.com/aylliote/senti-py#egg=senti-py

In [151]:
import senti-py-masterrr

SyntaxError: invalid syntax (<ipython-input-151-317e8402d430>, line 1)

In [157]:
from transformers import pipeline

# Allocate a pipeline for question-answering
question_answerer = pipeline('question-answering')
question_answerer({'question': 'What is the name of the repository ?', 'context': 'Pipeline has been included in the huggingface/transformers repository'})
#{'score': 0.30970096588134766, 'start': 34, 'end': 58, 'answer': 'huggingface/transformers'}


No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)
https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json not found in cache or force_download set to True, downloading to /Users/usuario/.cache/huggingface/transformers/tmpaddf5jpa


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json in cache at /Users/usuario/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
creating metadata file for /Users/usuario/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
loading configuration file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json from cache at /Users/usuario/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/pytorch_model.bin in cache at /Users/usuario/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
creating metadata file for /Users/usuario/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
loading weights file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/pytorch_model.bin from cache at /Users/usuario/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-c

...

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/vocab.txt in cache at /Users/usuario/.cache/huggingface/transformers/5e66e5f3d499a2fe36f07906eced96e223d283818c9a5fc17efd3e7bf4753711.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
creating metadata file for /Users/usuario/.cache/huggingface/transformers/5e66e5f3d499a2fe36f07906eced96e223d283818c9a5fc17efd3e7bf4753711.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /Users/usuario/.cache/huggingface/transformers/tmprugna7mj


Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/tokenizer.json in cache at /Users/usuario/.cache/huggingface/transformers/3b10caec238c150efa989f0540fb8a3876bc7e010fe50aa9d7a9d42c84fc0587.2b9a196704f2f183fe3f4b48d6e662dba8203fdcb3346bfa896831378edf6f97
creating metadata file for /Users/usuario/.cache/huggingface/transformers/3b10caec238c150efa989f0540fb8a3876bc7e010fe50aa9d7a9d42c84fc0587.2b9a196704f2f183fe3f4b48d6e662dba8203fdcb3346bfa896831378edf6f97
loading file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/vocab.txt from cache at /Users/usuario/.cache/huggingface/transformers/5e66e5f3d499a2fe36f07906eced96e223d283818c9a5fc17efd3e7bf4753711.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/tokenizer.json from cache at /Users/usuario/.cache/huggingface/transformers/3b10caec238c150efa989f0540fb8a3876bc7e010fe50aa9d7a9d42c84f

{'score': 0.3097020387649536,
 'start': 34,
 'end': 58,
 'answer': 'huggingface/transformers'}